In [1]:
import pickle
import pickle

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa
class TransformerModel(tf.keras.Model):
    def __init__(self, vocab_size, d_model, num_heads, ff_dim, num_transformer_blocks, output_dim, rate=0.1):
        # Call the constructor of the parent class (tf.keras.Model)
        super().__init__()  # This line is crucial for proper initialization

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.attention = tfa.layers.MultiHeadAttention(head_size=num_heads, num_heads=num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(d_model),
        ])

        self.rnn_layer = layers.GRU(16, return_sequences=True)
        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.pooling = layers.GlobalAveragePooling1D()
        self.dense = layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reshape(x, [-1, tf.shape(x)[1], self.embedding.output_dim])

        attention_output = self.attention([x, x, x])
        x = x + attention_output
        x = self.rnn_layer(x)
        x = self.layer_norm1(x)
        x = self.ffn(x)
        x = self.dropout1(x)
        x = x + attention_output
        x = self.layer_norm2(x)
        x = self.pooling(x)
        return self.dense(x)

c:\Users\me513\anaconda3\envs\gp\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
vocab_size = len(tokenizer.word_index) + 1
output_dim = 3
num_transformer_blocks = 2
d_model = 96
num_heads = 8
ff_dim = 128
dropout_rate = 0.1
learning_rate = 0.00022491452958964034
# Rebuild the model architecture
model = TransformerModel(vocab_size, d_model, num_heads, ff_dim, num_transformer_blocks, output_dim, rate=dropout_rate)

# Call the model with some dummy data to build its structure
dummy_input = tf.random.uniform((1, 10))  # A random tensor with shape (batch_size, input_length)
model(dummy_input)  # This will build the model

# Now load the weights
model.load_weights('transformer_weights.h5')


c:\Users\me513\anaconda3\envs\gp\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [4]:
import json
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
from flask import Flask, request, Response
import numpy as np

app = Flask(__name__)


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def prediction_test(review_padded):
    predictions = model.predict(review_padded)
    predicted_classes = predictions.argmax(axis=-1)
    predicted_classes = np.vectorize({2: -1, 1: 1, 0: 0}.get)(predicted_classes)
    return predicted_classes

def respond(review=" "):
    review_padded = tf.keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([review]), maxlen=100, padding='post', truncating='post'
    )
    pred = prediction_test(review_padded)
    return pred

@app.route('/predictAPI', methods=['POST'])
def create_page():
    input_sentence = request.json.get('inputs', "")  # Get the input sentence
    
    if not input_sentence:
        return Response(json.dumps({"error": "No input provided"}), status=400, mimetype='application/json')

    predicted = respond(input_sentence)  # Pass the input sentence directly to respond
    response_data = {'predicted': predicted}
    return Response(json.dumps(response_data, cls=NumpyEncoder), mimetype='application/json')

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.2:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 30ms/step


192.168.1.2 - - [21/Oct/2024 20:25:30] "POST /predictAPI HTTP/1.1" 200 -
